## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('train/train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
###Drop Nan Records
df=df.dropna()

In [5]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [6]:
## Get the Dependent features
y=df['label']

In [7]:
X.shape

(18285, 4)

In [8]:
y.shape

(18285,)

In [9]:
import tensorflow as tf

In [10]:
tf.__version__

'2.3.1'

In [11]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [12]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [13]:
messages=X.copy()

In [14]:
messages['text'][1]

'Ever get the feeling your life circles the roundabout rather than heads in a straight line toward the intended destination? [Hillary Clinton remains the big woman on campus in leafy, liberal Wellesley, Massachusetts. Everywhere else votes her most likely to don her inauguration dress for the remainder of her days the way Miss Havisham forever wore that wedding dress.  Speaking of Great Expectations, Hillary Rodham overflowed with them 48 years ago when she first addressed a Wellesley graduating class. The president of the college informed those gathered in 1969 that the students needed “no debate so far as I could ascertain as to who their spokesman was to be” (kind of the like the Democratic primaries in 2016 minus the   terms unknown then even at a Seven Sisters school). “I am very glad that Miss Adams made it clear that what I am speaking for today is all of us —  the 400 of us,” Miss Rodham told her classmates. After appointing herself Edger Bergen to the Charlie McCarthys and Mor

In [15]:
messages.reset_index(inplace=True)

In [16]:
import nltk
import re
from nltk.corpus import stopwords

In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Xavier\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    #print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['text'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [19]:
corpus[0]

'hous dem aid even see comey letter jason chaffetz tweet darrel lucu octob subscrib jason chaffetz stump american fork utah imag courtesi michael jolley avail creativ common licens apolog keith olbermann doubt worst person world week fbi director jame comey accord hous democrat aid look like also know second worst person well turn comey sent infam letter announc fbi look email may relat hillari clinton email server rank democrat relev committe hear comey found via tweet one republican committe chairmen know comey notifi republican chairmen democrat rank member hous intellig judiciari oversight committe agenc review email recent discov order see contain classifi inform long letter went oversight committe chairman jason chaffetz set polit world ablaz tweet fbi dir inform fbi learn exist email appear pertin investig case reopen jason chaffetz jasoninthehous octob cours know case comey actual say review email light unrel case know anthoni weiner sext teenag appar littl thing fact matter ch

In [20]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr[0]

[2582,
 4402,
 2004,
 1389,
 4576,
 1346,
 1096,
 206,
 4452,
 3155,
 134,
 370,
 2873,
 887,
 206,
 4452,
 3507,
 264,
 2834,
 842,
 2245,
 220,
 4428,
 3750,
 773,
 136,
 3355,
 3498,
 1379,
 904,
 1718,
 1663,
 827,
 3444,
 3224,
 561,
 1364,
 1764,
 3331,
 1346,
 624,
 2582,
 4262,
 2004,
 2126,
 4369,
 3268,
 2296,
 3054,
 827,
 3444,
 219,
 2742,
 1346,
 4133,
 3341,
 1096,
 1905,
 1364,
 2126,
 2232,
 491,
 229,
 4758,
 948,
 2232,
 2448,
 3632,
 4262,
 1213,
 3596,
 239,
 1346,
 4981,
 2811,
 3155,
 3321,
 1152,
 3596,
 4723,
 2296,
 1346,
 4183,
 1152,
 4723,
 4262,
 3632,
 1282,
 2582,
 989,
 1694,
 614,
 3596,
 1053,
 1438,
 2232,
 2729,
 233,
 3995,
 4576,
 2138,
 2962,
 3776,
 963,
 1096,
 3162,
 614,
 3596,
 1753,
 206,
 4452,
 2584,
 2367,
 3224,
 3299,
 3155,
 1364,
 716,
 3776,
 1364,
 132,
 488,
 2232,
 3825,
 911,
 2073,
 853,
 2689,
 206,
 4452,
 498,
 2873,
 2644,
 2296,
 853,
 1346,
 4695,
 4983,
 1438,
 2232,
 1306,
 1464,
 853,
 2296,
 831,
 4794,
 1774,
 349,
 

### Embedding Representation

In [21]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[ 109 1380  491 ... 3060 2582  203]
 [4127 1719  531 ... 3827 4291  267]
 [2789 2923 2389 ... 4262 4918  764]
 ...
 [3939  482 3882 ... 2772 1148 1238]
 [1711 1144 2463 ... 2169 2097 3614]
 [ 869 3468  183 ... 2925 1350 4088]]


In [22]:
embedded_docs[0]

array([ 109, 1380,  491, 2296,  261, 2660, 3830, 4402,  939,  545, 2813,
       4754,  203, 3055, 1431, 3947,  134, 3060, 2582,  203])

In [23]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [24]:
len(embedded_docs),y.shape

(18285, (18285,))

In [25]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [26]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [28]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 5s 28ms/step - loss: 0.4262 - accuracy: 0.7843 - val_loss: 0.3333 - val_accuracy: 0.8497
Epoch 2/10
192/192 [==============================] - 5s 24ms/step - loss: 0.2480 - accuracy: 0.8970 - val_loss: 0.3371 - val_accuracy: 0.8515
Epoch 3/10
192/192 [==============================] - 5s 24ms/step - loss: 0.1892 - accuracy: 0.9272 - val_loss: 0.3554 - val_accuracy: 0.8489
Epoch 4/10
192/192 [==============================] - 5s 24ms/step - loss: 0.1371 - accuracy: 0.9515 - val_loss: 0.4189 - val_accuracy: 0.8391
Epoch 5/10
192/192 [==============================] - 5s 24ms/step - loss: 0.0994 - accuracy: 0.9645 - val_loss: 0.4814 - val_accuracy: 0.8365
Epoch 6/10
192/192 [==============================] - 5s 24ms/step - loss: 0.0651 - accuracy: 0.9776 - val_loss: 0.5919 - val_accuracy: 0.8366
Epoch 7/10
192/192 [==============================] - 5s 24ms/step - loss: 0.0451 - accuracy: 0.9851 - val_loss: 0.7002 - val_accuracy: 0.8247

### Adding Dropout 

In [ ]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Performance Metrics And Accuracy

In [29]:
y_pred=model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [30]:
from sklearn.metrics import confusion_matrix

In [31]:
confusion_matrix(y_test,y_pred)

array([[2789,  630],
       [ 460, 2156]], dtype=int64)

In [32]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8193869096934548